# LangGraph Sidekick – Week 4 Final Assignment

## Author: Bharat Puri

## Objective:

Extend and personalize your LangGraph-based “Sidekick” by adding custom tools, persistent SQL memory, and a planning mechanism.
The assistant should demonstrate autonomy, context retention, and the ability to use tools intelligently.

In [3]:
!pip install -U -q imapclient langchain==1.0.2 langchain-openai==1.0.1 langchain-chroma==1.0.0 langchain-community==0.4 langchain-core==1.0.0 langchain-text-splitters==1.0.0 langchain-huggingface==1.0.0 langchain-classic==1.0.0 chromadb==1.2.1 sentence-transformers==5.1.2

In [4]:
!pip install -q langgraph sqlite-utils gradio


In [10]:
# -------------------------------------------------------------
# 1. Imports
# -------------------------------------------------------------
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langgraph.graph import StateGraph, END
import gradio as gr
import os

In [12]:
# -------------------------------------------------------------
# 2. Load Environment Variables
# -------------------------------------------------------------
load_dotenv(override=True)

# Required in .env:
# OPENAI_API_KEY=sk-xxxx

if "OPENAI_API_KEY" not in os.environ:
    raise EnvironmentError("❌ OPENAI_API_KEY not found. Please check your .env file.")
else:
    print("✅ Environment loaded successfully.")

✅ Environment loaded successfully.


In [13]:
# -------------------------------------------------------------
# 3. Define Custom Tools
# -------------------------------------------------------------
@tool
def search_docs(query: str, folder: str = "./docs"):
    """
    Search .txt files in a folder for a given keyword.
    """
    results = []
    if not os.path.exists(folder):
        return f"❌ Folder '{folder}' not found."

    for file in os.listdir(folder):
        if file.endswith(".txt"):
            with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
                text = f.read().lower()
                if query.lower() in text:
                    results.append(file)
    return results or "No matches found."



In [14]:
@tool
def run_python(expression: str):
    """
    Safely evaluate simple Python expressions.
    """
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"✅ Result: {result}"
    except Exception as e:
        return f"❌ Error: {e}"

tools = [search_docs, run_python]

In [25]:
# -------------------------------------------------------------
# 4. Persistent SQL Memory Setup
# -------------------------------------------------------------
memory = SQLChatMessageHistory(
    connection_string="sqlite:///sidekick_memory.db",
    session_id="bharat_user"
)

from langchain_core.messages import HumanMessage, AIMessage

def remember_message(role: str, message: str):
    """
    Stores user/assistant messages in persistent SQL memory (LangChain v1.0+ format).
    """
    if role == "user":
        memory.add_message(HumanMessage(content=message))
    elif role == "assistant":
        memory.add_message(AIMessage(content=message))


In [26]:
# -------------------------------------------------------------
# 5. Initialize LangGraph Agents
# -------------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

planner_prompt = ChatPromptTemplate.from_template("""
You are the PLANNER agent. Analyze the user request and describe what tasks should be performed.
User request: {user_input}
""")

worker_prompt = ChatPromptTemplate.from_template("""
You are the WORKER agent. Follow the planner's instruction carefully.
Instruction: {instruction}
""")

def planner_node(state):
    plan = llm.invoke(planner_prompt.format(user_input=state["user_input"]))
    return {"instruction": plan.content}

def worker_node(state):
    result = llm.invoke(worker_prompt.format(instruction=state["instruction"]))
    return {"result": result.content}

In [27]:
# -------------------------------------------------------------
# 6. Build LangGraph Workflow (with state_schema)
# -------------------------------------------------------------
from typing import TypedDict

# Define the structure of your graph state
class GraphState(TypedDict):
    user_input: str
    instruction: str
    result: str

# Initialize the state graph with schema
graph = StateGraph(GraphState)

# Add planner and worker nodes
graph.add_node("planner", planner_node)
graph.add_node("worker", worker_node)

# Connect the flow planner → worker → END
graph.add_edge("planner", "worker")
graph.add_edge("worker", END)

# Define the entry point
graph.set_entry_point("planner")

# Compile the graph
compiled_graph = graph.compile()


In [28]:
# -------------------------------------------------------------
# 7. Clarifying Questions
# -------------------------------------------------------------
def ask_clarifying_questions(user_input: str) -> str:
    """
    Generates clarifying questions to ensure task understanding.
    """
    clarifications = [
        f"Can you clarify what exactly you want me to do with: '{user_input}'?",
        "Should I use a specific folder or tool for this task?",
        "What level of detail do you expect in the output?"
    ]
    return "\n".join(clarifications)

In [29]:
# -------------------------------------------------------------
# 8. Core Assistant Logic
# -------------------------------------------------------------
def run_sidekick(user_input: str):
    """
    Executes the LangGraph assistant flow.
    """
    print("🤖 Step 1: Asking clarifying questions...")
    clarifications = ask_clarifying_questions(user_input)

    print("🧠 Step 2: Planning and executing tasks...")
    remember_message("user", user_input)
    result = compiled_graph.invoke({"user_input": user_input})
    remember_message("assistant", str(result))

    return f"### Clarifications\n{clarifications}\n\n### Response\n{result['result']}"


In [21]:
# -------------------------------------------------------------
# 9. Gradio Interface
# -------------------------------------------------------------
def interface_fn(user_input):
    return run_sidekick(user_input)

iface = gr.Interface(
    fn=interface_fn,
    inputs=gr.Textbox(
        label="Enter your request",
        placeholder="e.g., Search for AI papers or evaluate 3*8+5",
        lines=2,
    ),
    outputs=gr.Markdown(label="AI Sidekick Response"),
    title="🧠 LangGraph AI Sidekick with SQL Memory",
    description=(
        "A personalized assistant powered by LangGraph and OpenAI.\n\n"
        "💡 Supports planning, task execution, and SQL-based persistent memory."
    ),
)

In [32]:
# -------------------------------------------------------------
# 9b. Enhanced Gradio Interface with Memory Viewer
# -------------------------------------------------------------
import pandas as pd

def view_memory():
    """
    Parses and displays recent messages from SQL memory in a chat-style format.
    Works with latest LangChain SQLChatMessageHistory schema.
    """
    import sqlite3, json

    try:
        conn = sqlite3.connect("sidekick_memory.db")
        cursor = conn.cursor()
        cursor.execute("SELECT message FROM message_store ORDER BY id DESC LIMIT 10")
        rows = cursor.fetchall()
        conn.close()

        if not rows:
            return "🗂️ No messages found in memory."

        messages_md = "### 🧠 Conversation Memory\n\n"

        for row in reversed(rows):
            try:
                msg_obj = json.loads(row[0])
                msg_type = msg_obj.get("type", "")
                msg_data = msg_obj.get("data", {})
                content = msg_data.get("content", "")
                if msg_type == "human":
                    messages_md += f"🧑 **User:** {content}\n\n"
                elif msg_type == "ai":
                    messages_md += f"🤖 **Assistant:** {content}\n\n"
                else:
                    messages_md += f"💬 **{msg_type.capitalize()}:** {content}\n\n"
            except Exception as inner_e:
                messages_md += f"⚠️ [Unreadable Entry] {row[0][:80]}...\n\n"

        return messages_md

    except Exception as e:
        return f"⚠️ Error reading memory: {e}"


# Main tab – Chat interface
with gr.Blocks() as iface:
    gr.Markdown("## 🧠 LangGraph AI Sidekick with SQL Memory")
    gr.Markdown("A personalized assistant powered by LangGraph and OpenAI.\n\n"
                "💡 Supports planning, task execution, and SQL-based persistent memory.")
    
    with gr.Tab("💬 Chat"):
        input_box = gr.Textbox(
            label="Enter your request",
            placeholder="e.g., Search for AI papers or evaluate 3*8+5",
            lines=2,
        )
        output_box = gr.Markdown(label="AI Sidekick Response")
        submit_btn = gr.Button("Submit", variant="primary")
        submit_btn.click(fn=interface_fn, inputs=input_box, outputs=output_box)

    with gr.Tab("🗂️ Memory Viewer"):
        gr.Markdown("### Recent Conversation Memory")
        memory_output = gr.Markdown()
        refresh_btn = gr.Button("🔄 Refresh Memory")
        refresh_btn.click(fn=view_memory, inputs=None, outputs=memory_output)

In [33]:
# -------------------------------------------------------------
# 10. Launch App
# -------------------------------------------------------------
iface.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


🤖 Step 1: Asking clarifying questions...
🧠 Step 2: Planning and executing tasks...
